In [2]:
import os
import datetime
import csv

def ctime2utcstr(t):
    dt = datetime.datetime.utcfromtimestamp(t)
    return datetime.datetime.strftime(dt, "%c")

def filename2ctime(filename):
    dtstr = filename.split("_",2)[2].split(".")[0]
    dtobj = datetime.datetime.strptime(dtstr, "%Y-%m-%d_%H-%M-%S")
    dtobjutc = dtobj.replace(tzinfo=datetime.timezone.utc)
    return dtobjutc.timestamp()

def fileModTime(filename):
    return os.path.getmtime(filename)  # UTC time as a float

def getWGScoord(fname):
    with open(fname, 'r') as f:
        # skip the first 11 lines
        for _ in range(11):
            f.readline()
        # the last line is a string with coordinates
        coords = f.readline()
        f.close()
    return coords

# objective: create a list of (ctime, time string, location) for all files in a given directory
def parseLocFiles(directory):
    filenames = os.listdir(directory)
    locs = []  # empty list to hold results
    for filename in filenames:
        fullpath = os.path.join(directory, filename)
        coords = getWGScoord(fullpath)
        t = fileModTime(fullpath)
        tstr = ctime2utcstr(t)
        locs.append((t, tstr, coords))        
    return locs

# objective: create a list of (ctime, time string, file) for all files in a given directory
def captureTimeDataFiles(directory, basedir=None):
    filenames = os.listdir(directory)
    files = []
    for filename in filenames:
        fullpath = os.path.join(directory, filename)
        tname = filename2ctime(filename)
        tstr = ctime2utcstr(tname)
        if basedir:
            relpath = os.path.relpath(fullpath, basedir)
            files.append((tname,tstr,relpath))
        else:
            files.append((tname,tstr,fullpath))
    return files

# print("locations:")
locationdir = "/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/location_data_archive"
locations = parseLocFiles(locationdir)
# print(*locations, sep="\n")
# print()

# make a list of all the directories

basedir = "/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data"
devs = ["eggtart", "bananacream"]
tests = ["outdoor_11june", "outdoor_13june"]

datadirs = []

for test in tests:
    for dev in devs:
        d = os.path.join(basedir, test, dev)
        datadirs.append(d)

datafiles = []
for datadir in datadirs:
    datafiles.extend(captureTimeDataFiles(datadir, basedir))
# print("datafiles:")
# print(*datafiles, sep="\n")
# print()

# combine lists, then sort them
combList = datafiles + locations
combList = sorted(combList, key=lambda x:x[0])
# print(*combList, sep="\n")

with open("sdr_data_location.csv", 'w') as f:
    writer = csv.writer(f, quotechar="'", quoting=csv.QUOTE_NONNUMERIC)
    for entry in combList:
        writer.writerow(entry)

/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/eggtart
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_11june/bananacream
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/eggtart
/media/adwait/44EE36CDEE36B6D0/workspace/sdr_data/data/outdoor_13june/bananacream
